In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from selenium import webdriver

In [70]:
driver = webdriver.Chrome()
# Open the URL of the webpage
url = "https://store.steampowered.com/search/?sort_by=_ASC&ignore_preferences=1&category1=998&supportedlang=english"
driver.get(url)

# Automatically scroll the page
prev_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # let content load

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == prev_height:
        break  # No new content loaded
    prev_height = new_height


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
}
# helper to fetch a URL with a random sleep
def fetch(url, headers=None):
    # sleep between 1 and 3 seconds
    time.sleep(random.uniform(1, 3))
    return requests.get(url, headers=headers)


url = 'https://store.steampowered.com/search/?sort_by=_ASC&ignore_preferences=1&category1=998&supportedlang=english'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
game_elements = soup.select('.search_result_row')

game_name = []
game_date_released = []
game_price = []
game_review_1 = []
game_review_2 = []
game_discount_price = []
game_discount_pct = []
game_links = []
game_description = []
game_tags = []

# --- Prepare lists for developer, publisher, and image ---
developers = []    # List of developers for all games
publishers = []    # List of publishers for all games
game_images = []   # List of game header image URLs

start = 0
max_results = 10000
for game in game_elements[:max_results]:
    print(game.select_one('.title').text.strip())
    game_name.append(game.select_one('.title').text.strip() if game.select_one('.title') else None)
    game_date_released.append(game.select_one('.search_released').text.strip() if game.select_one('.search_released') else None)
    game_price.append(game.select_one('.discount_final_price').text.strip() if game.select_one('.discount_final_price') else None)
    
    review_elem = game.select_one('.search_reviewscore span')
    if review_elem:
        game_review_1.append(review_elem['data-tooltip-html'].split('<br>')[0])
        game_review_2.append(review_elem['data-tooltip-html'].split('<br>')[1])
    else:
        game_review_1.append(None)
        game_review_2.append(None)
    
    discounted_price_elem = game.select_one('.discount_final_price')
    discount_pct_elem = game.select_one('.discount_pct')
    if discount_pct_elem:
        game_discount_pct.append(discount_pct_elem.text.strip())
        game_discount_price.append(discounted_price_elem.text.strip())
    else:
        game_discount_price.append(None)
        game_discount_pct.append(None)
        
    game_links.append(game.get('href'))

for url_2 in game_links:
    response_2 = requests.get(url_2)
    soup_2 = BeautifulSoup(response_2.text, 'html.parser')
    
    # Extract game description safely
    game_ = soup_2.select('.game_description_snippet')
    game_description.append(game_[0].text.strip() if game_ else None)
    
    tags = soup_2.select('a.app_tag')
    game_tags.append([tag.text.strip() for tag in tags] if tags else [None])
    
    # --- Extract Game Image ---
    img_elem = soup_2.find('img', class_='game_header_image_full')
    img_url = img_elem['src'] if img_elem and img_elem.has_attr('src') else None
    game_images.append(img_url)
    
    dev_row_blocks = soup_2.select('div.dev_row')
    for row in dev_row_blocks:
        header = row.find('b')
        if header:
            header_text = header.text.strip().lower()
            if 'developer' in header_text:
                    dev_links = row.find_all('a')
                    dev = [a.text.strip() for a in dev_links]
            elif 'publisher' in header_text:
                pub_links = row.find_all('a')
                pub = [a.text.strip() for a in pub_links]

    developers.append(dev if dev else [None])
    publishers.append(pub if pub else [None])
            
    
driver.quit()
    
    # learn from here - https://medium.com/@irfanbasyar.ib/how-to-scrape-data-from-a-webpage-with-infinite-scrolling-using-python-and-beautifulsoup-f3c39e52ae8 



Counter-Strike 2
Schedule I
Clair Obscur: Expedition 33
The Elder Scrolls IV: Oblivion Remastered
R.E.P.O.
Destiny 2
Stellaris
War Thunder
Marvel Rivals
EA SPORTS FC™ 25
Apex Legends™
Rust
Crusader Kings III
Dead by Daylight
Baldur's Gate 3
Warframe
The Sims™ 4
The Elder Scrolls® Online
Forza Horizon 5
Tom Clancy's Rainbow Six® Siege
V Rising
Drive Beyond Horizons
Split Fiction
Once Human
Train Sim World® 5
Blue Prince
RuneScape: Dragonwilds
THRONE AND LIBERTY
The Last of Us™ Part II Remastered
NBA 2K25
Hearts of Iron IV
HELLDIVERS™ 2
inZOI
STAR WARS Jedi: Survivor™
ELDEN RING
Tempest Rising
Arma Reforger
Age of Empires II: Definitive Edition
Cyberpunk 2077
Kingdom Come: Deliverance II
DARK SOULS™ III
Team Fortress 2
FINAL FANTASY XIV Online
Nine Sols
Path of Exile 2
Deadzone: Rogue
Assassin’s Creed Shadows
Euro Truck Simulator 2
Sea of Thieves: 2025 Edition
PowerWash Simulator
Red Dead Redemption
The Elder Scrolls V: Skyrim Special Edition
Monster Hunter Wilds
Black Desert
RimWorld
Re

In [71]:
df = pd.DataFrame({
    'Name':game_name, 
    'Description':game_description, 
    'Price':game_price,
    'Release Date': game_date_released,
    'Description': game_description,
    'Discounted Price': game_discount_price,
    'Discount Percentage': game_discount_pct,
    'Review': game_review_1,
    'Review_2': game_review_2,
    'Links': game_links,
    'Tags': game_tags,
    'Developers': developers,
    'Publishers': publishers,
    'Images': game_images})

In [74]:
df.tail(10)

,Name,Description,Price,Release Date,Discounted Price,Discount Percentage,Review,Review_2,Links,Tags,Developers,Publishers,Images
9990,Westerado: Double Barreled,Westerado: Double Barreled. When your family i...,£10.99,"16 Apr, 2015",None,None,Very Positive,"92% of the 1,378 user reviews for this game ar...",https://store.steampowered.com/app/275200/West...,"[Western, Pixel Graphics, Open World, Adventur...",[Ostrich Banditos],[Ostrich Banditos],https://shared.fastly.steamstatic.com/store_it...
9991,Mail Mole + 'Xpress Deliveries,Mail Mole is a charming 3D platformer featurin...,£12.99,"3 Mar, 2021",None,None,Very Positive,90% of the 162 user reviews for this game are ...,https://store.steampowered.com/app/1273540/Mai...,"[3D Platformer, Platformer, Collectathon, Cute...","[Talpa Games, Undercoders]",[Undercoders],https://shared.fastly.steamstatic.com/store_it...
9992,Hourglass,In Hourglass you can create a time-clone of yo...,£15.99,"22 Oct, 2021",None,None,Very Positive,88% of the 295 user reviews for this game are ...,https://store.steampowered.com/app/1212410/Hou...,"[Puzzle, Puzzle Platformer, First-Person, Atmo...",[Cyberwave],[Cyberwave],https://shared.fastly.steamstatic.com/store_it...
9993,Hadaka Shitsuji - Naked Butlers,A butler-bullying Visual Novel for people who ...,£27.79,"28 Feb, 2019",None,None,Very Positive,86% of the 110 user reviews for this game are ...,https://store.steampowered.com/app/1031020/Had...,"[Sexual Content, Nudity, LGBTQ+, Visual Novel,...",[MADA Labo],[MangaGamer],https://shared.fastly.steamstatic.com/store_it...
9994,XIII - Classic,"You are the victim of a conspiracy! The FBI, C...",£10.99,"11 Jun, 2020",None,None,Very Positive,"90% of the 2,031 user reviews for this game ar...",https://store.steampowered.com/app/1170760/XII...,"[Action, FPS, Classic, Comic Book, Stealth, Vi...",[Ubisoft Paris Studio],[Microids],https://shared.fastly.steamstatic.com/store_it...
9995,Cinderella Phenomenon: Evermore,What happens after Happily Ever After…?,£7.19,"13 Jan, 2021",None,None,Very Positive,96% of the 189 user reviews for this game are ...,https://store.steampowered.com/app/1496180/Cin...,"[Visual Novel, Romance, Otome, Dating Sim, Fan...",[Dicesuki],[Dicesuki],https://shared.fastly.steamstatic.com/store_it...
9996,PlayForm: Human Dynamics,PlayForm is an experimental physics sandbox. P...,£15.49,"27 Jul, 2022",None,None,Very Positive,90% of the 137 user reviews for this game are ...,https://store.steampowered.com/app/1729300/Pla...,"[Sandbox, Simulation, Physics, Parkour, Funny,...",[Code}{atch],[Code}{atch],https://shared.fastly.steamstatic.com/store_it...
9997,Tiny Civilization,This is a historical simulation game that prom...,£1.69,"1 Jan, 2023",None,None,Very Positive,"94% of the 1,955 user reviews for this game ar...",https://store.steampowered.com/app/2230280/Tin...,"[Strategy, God Game, Grand Strategy, Tabletop,...",[喜人一个],[喜人一个],https://shared.fastly.steamstatic.com/store_it...
9998,Rainy Season,This is a story about an ordinary family spend...,£2.89,"19 Aug, 2020",None,None,Overwhelmingly Positive,95% of the 502 user reviews for this game are ...,https://store.steampowered.com/app/1094420/Rai...,"[Indie, Casual, Atmospheric, Relaxing, Anime, ...",[Inasa Fujio],[Inasa Fujio],https://shared.fastly.steamstatic.com/store_it...
9999,X-Fitness,X-Fitness is a fun and innovative way to enjoy...,£13.17,"3 Dec, 2020",£13.17,-33%,Very Positive,85% of the 118 user reviews for this game are ...,https://store.steampowered.com/app/1314590/XFi...,"[Sports, Action, Casual, Rhythm, Music, VR]",[NiVision],[NiVision],https://shared.fastly.steamstatic.com/store_it...


In [79]:
df['Name'].nunique()

9995

In [81]:
len(df['Name']) - df['Name'].nunique()

5

In [82]:
df[df.duplicated(subset='Name', keep=False)]

,Name,Description,Price,Release Date,Discounted Price,Discount Percentage,Review,Review_2,Links,Tags,Developers,Publishers,Images
37,Age of Empires II: Definitive Edition,"42 Civilizations, 229 Campaign Missions, Singl...",£9.09,"14 Nov, 2019",£9.09,-65%,Very Positive,"94% of the 141,248 user reviews for this game ...",https://store.steampowered.com/app/813780/Age_...,"[Strategy, RTS, City Builder, Multiplayer, Bas...","[World's Edge, Forgotten Empires, Tantalus Med...",[Xbox Game Studios],https://shared.fastly.steamstatic.com/store_it...
1218,Age of Empires II: Definitive Edition,None,£9.09,"14 Nov, 2019",£9.09,-65%,Very Positive,"94% of the 142,888 user reviews for games in t...",https://store.steampowered.com/sub/248721/?snr...,[None],[RageSquid],[No More Robots],None
8360,Red Blue Cell,This is a retro barrage flying shooting game. ...,£2.71,"10 Jan, 2025",£2.71,-20%,Positive,100% of the 16 user reviews for this game are ...,https://store.steampowered.com/app/3262220/Red...,"[Bullet Hell, 1990's, Arcade, Pixel Graphics, ...","[Liao Kai, LKK GAME]",[Liao Kai],https://shared.fastly.steamstatic.com/store_it...
9249,Secret Love Temple,Two lovely elf women find passion and pleasure...,£4.99,"31 Mar, 2025",None,None,Positive,93% of the 16 user reviews for this game are p...,https://store.steampowered.com/app/3563430/Sec...,"[Sexual Content, Nudity, NSFW, Mature, Movie, ...",[TheBigO Games],[EroBits],https://shared.fastly.steamstatic.com/store_it...
9250,Secret Love Temple,Two lovely elf women find passion and pleasure...,£4.99,"31 Mar, 2025",None,None,Positive,93% of the 16 user reviews for this game are p...,https://store.steampowered.com/app/3563430/Sec...,"[Sexual Content, Nudity, NSFW, Mature, Movie, ...",[TheBigO Games],[EroBits],https://shared.fastly.steamstatic.com/store_it...
9449,VirtuaVerse,VirtuaVerse is a challenging old school cyberp...,£12.99,"12 May, 2020",None,None,Very Positive,82% of the 928 user reviews for this game are ...,https://store.steampowered.com/app/1019310/Vir...,"[Cyberpunk, Point & Click, Old School, Dystopi...",[Theta Division],[Blood Music],https://shared.fastly.steamstatic.com/store_it...
9450,VirtuaVerse,VirtuaVerse is a challenging old school cyberp...,£12.99,"12 May, 2020",None,None,Very Positive,82% of the 928 user reviews for this game are ...,https://store.steampowered.com/app/1019310/Vir...,"[Cyberpunk, Point & Click, Old School, Dystopi...",[Theta Division],[Blood Music],https://shared.fastly.steamstatic.com/store_it...
9681,Red Blue Cell,This is a retro barrage flying shooting game. ...,£2.71,"10 Jan, 2025",£2.71,-20%,Positive,100% of the 16 user reviews for this game are ...,https://store.steampowered.com/app/3262220/Red...,"[Bullet Hell, 1990's, Arcade, Pixel Graphics, ...","[Liao Kai, LKK GAME]",[Liao Kai],https://shared.fastly.steamstatic.com/store_it...
9699,White Day 2: The Flower That Tells Lies - Comp...,Return to Yeondu High School and uncover the d...,£13.74,"15 Feb, 2023",£13.74,-45%,Mixed,46% of the 568 user reviews for this game are ...,https://store.steampowered.com/app/2078040/Whi...,"[Action-Adventure, Sequel, Exploration, Puzzle...",[Rootnstudio Ltd.],[PQube],https://shared.fastly.steamstatic.com/store_it...
9700,White Day 2: The Flower That Tells Lies - Comp...,Return to Yeondu High School and uncover the d...,£13.74,"15 Feb, 2023",£13.74,-45%,Mixed,46% of the 568 user reviews for this game are ...,https://store.steampowered.com/app/2078040/Whi...,"[Action-Adventure, Sequel, Exploration, Puzzle...",[Rootnstudio Ltd.],[PQube],https://shared.fastly.steamstatic.com/store_it...


In [77]:
df.to_csv('games_data.csv')